## 유전 알고리즘(Genetic Algorithm)

- 유전 알고리즘은 진화론의 교차, 돌연변이 등의 아이디어를 차용하여 세대를 거듭하며 다양성을 줄여나가며 더 나은 해를 찾아가는 최적화 기법

1. 초기 모집단
2. 적합도 계산
3. 부모 선택
4. 재생산(교차)
5. 재생산(돌연변이)
6. 다음 세대 채우기

```java
Function GENETIC_ALGORITHM(problem, population_size, mutation_rate, max_generations)
  population ← problem.random_initial_population(population_size)
  best_solution ← null
  best_fitness ← 0
  
  For generation = 1 to max_generations do
    fitness_scores ← [problem.fitness(individual) for individual in population]
    
    max_fitness_in_gen ← max(fitness_scores)
    if max_fitness_in_gen > best_fitness then
      best_fitness ← max_fitness_in_gen
      best_solution ← population[fitness_scores.index(max_fitness_in_gen)]
    
    new_population ← []
    For i = 1 to population_size // 2 do
      parent1 ← problem.select(population, fitness_scores)
      parent2 ← problem.select(population, fitness_scores)
      
      child1, child2 ← problem.crossover(parent1, parent2)
      
      if (random() < mutation_rate) then child1 ← problem.mutate(child1)
      if (random() < mutation_rate) then child2 ← problem.mutate(child2)
      
      new_population.append(child1)
      new_population.append(child2)
    
    population ← new_population
    
  return best_solution, best_fitness
``

In [4]:
import random

class KnapsackGA:
    def __init__(self, items, capacity):
        self.items = items  # 아이템 목록 (이름, 가치, 무게) 튜플로 구성
        self.capacity = capacity  # 배낭 용량
        self.chromosome_size = len(items)  # 염색체 크기 (아이템의 개수)

    # 개체 적합도 계산 (총 가치 반환)
    def fitness(self, chromosome):
        total_value = 0
        total_weight = 0
        for i in range(self.chromosome_size):
            if chromosome[i] == 1:  # 아이템이 선택된 경우
                _, value, weight = self.items[i]
                total_value += value
                total_weight += weight
        if total_weight > self.capacity:  # 배낭 용량 초과 시 적합도는 0
            return 0
        return total_value

    # 초기 모집단 생성
    def random_initial_population(self, population_size):
        population = []
        for _ in range(population_size):
            chromosome = [random.randint(0, 1) for _ in range(self.chromosome_size)]
            population.append(chromosome)
        return population

    # 선택 연산 (룰렛 휠 선택)
    def select(self, population, fitness_scores):
        total_fitness = sum(fitness_scores) # 전체 적합도 합계
        
        if total_fitness == 0:
            return random.choice(population)  # 적합도가 모두 0일 경우 임의의 부모 반환
        
        pick = random.uniform(0, total_fitness) # 0에서 total_fitness 사이의 무작위 값 선택 (룰렛 휠에서의 선택)
        
        current = 0
        # 적합도를 누적하여 룰렛 휠에서 선택된 개체 찾기
        for i in range(len(population)):  
            current += fitness_scores[i]  # 적합도를 누적
            if current > pick:  # 무작위 값이 현재 누적된 적합도보다 작으면 개체 선택
                return population[i]
        return population[0] # 기본으로 첫 번째 개체 반환

    # 교차 연산
    def crossover(self, parent1, parent2):
        i = random.randint(1, self.chromosome_size - 1)
        child1 = parent1[:i] + parent2[i:]
        child2 = parent2[:i] + parent1[i:]
        return child1, child2

    # 돌연변이 연산
    def mutate(self, chromosome, mutation_rate):
        for i in range(self.chromosome_size):
            if random.random() < mutation_rate:
                chromosome[i] = 1 - chromosome[i]  # 유전자를 0에서 1로, 또는 1에서 0으로 뒤집음


In [1]:
def genetic_algorithm(knapsack, population_size, mutation_rate, max_generations):
    # 초기 모집단 생성
    population = knapsack.random_initial_population(population_size)
    best_solution = None
    best_fitness = 0

    for generation in range(max_generations):
        # 적합도 계산
        fitness_scores = [knapsack.fitness(chromosome) for chromosome in population]

        # 최고 적합도 개체 확인
        max_fitness_in_gen = max(fitness_scores)
        if max_fitness_in_gen > best_fitness:
            best_fitness = max_fitness_in_gen
            best_solution = population[fitness_scores.index(max_fitness_in_gen)]

        print(f"세대 {generation + 1}: 최고 적합도 = {best_fitness}")

        # 새로운 세대 생성
        new_population = []
        for _ in range(population_size // 2):
            # 부모 선택(룰렛 휠 전략)
            parent1 = knapsack.select(population, fitness_scores)
            parent2 = knapsack.select(population, fitness_scores)
            # 개체 재생산: 교차
            child1, child2 = knapsack.crossover(parent1, parent2)
            # 개체 재생산: 돌연변이
            knapsack.mutate(child1, mutation_rate)
            knapsack.mutate(child2, mutation_rate)
            # 다음 세대 채우기
            new_population.append(child1)
            new_population.append(child2)
        # 세대 교체
        population = new_population

    return best_solution, best_fitness


In [2]:
items = [
    ("책", 45, 1.5),
    ("노트북", 350, 2.8),
    ("물병", 35, 0.9),
    ("필통", 20, 0.6),
    ("간식", 75, 0.4),
    ("이어폰", 100, 0.3),
    ("공책", 30, 1.1),
    ("전공책", 150, 2.2),
    ("펜", 8, 0.2),
    ("자", 12, 0.4),
    ("계산기", 90, 0.7),
    ("마커", 25, 0.3),
    ("USB 메모리", 55, 0.1),
    ("파일", 40, 0.8),
    ("가위", 22, 0.5)
]

capacity = 8

In [5]:
knapsack = KnapsackGA(items, capacity)

# 유전 알고리즘 설정
population_size = 10  # 모집단 크기
mutation_rate = 0.05  # 돌연변이 확률
max_generations = 150  # 최대 세대 수

# 유전 알고리즘 실행
best_solution, best_fitness = genetic_algorithm(knapsack, population_size, mutation_rate, max_generations)

print(f"\n최적 해: {best_solution}, 총 가치: {best_fitness}")


세대 1: 최고 적합도 = 565
세대 2: 최고 적합도 = 565
세대 3: 최고 적합도 = 568
세대 4: 최고 적합도 = 568
세대 5: 최고 적합도 = 568
세대 6: 최고 적합도 = 568
세대 7: 최고 적합도 = 568
세대 8: 최고 적합도 = 568
세대 9: 최고 적합도 = 673
세대 10: 최고 적합도 = 673
세대 11: 최고 적합도 = 678
세대 12: 최고 적합도 = 753
세대 13: 최고 적합도 = 753
세대 14: 최고 적합도 = 753
세대 15: 최고 적합도 = 770
세대 16: 최고 적합도 = 770
세대 17: 최고 적합도 = 770
세대 18: 최고 적합도 = 770
세대 19: 최고 적합도 = 770
세대 20: 최고 적합도 = 770
세대 21: 최고 적합도 = 770
세대 22: 최고 적합도 = 770
세대 23: 최고 적합도 = 770
세대 24: 최고 적합도 = 770
세대 25: 최고 적합도 = 770
세대 26: 최고 적합도 = 770
세대 27: 최고 적합도 = 770
세대 28: 최고 적합도 = 780
세대 29: 최고 적합도 = 867
세대 30: 최고 적합도 = 867
세대 31: 최고 적합도 = 867
세대 32: 최고 적합도 = 867
세대 33: 최고 적합도 = 867
세대 34: 최고 적합도 = 867
세대 35: 최고 적합도 = 867
세대 36: 최고 적합도 = 867
세대 37: 최고 적합도 = 867
세대 38: 최고 적합도 = 867
세대 39: 최고 적합도 = 867
세대 40: 최고 적합도 = 867
세대 41: 최고 적합도 = 867
세대 42: 최고 적합도 = 867
세대 43: 최고 적합도 = 867
세대 44: 최고 적합도 = 867
세대 45: 최고 적합도 = 867
세대 46: 최고 적합도 = 867
세대 47: 최고 적합도 = 867
세대 48: 최고 적합도 = 867
세대 49: 최고 적합도 = 867
세대 50: 최고 적합도 = 867
세대 51: 최고